# Numerične metode v tehniki
## Diferencialne enačbe - 2. del - BVP

- **avtor**: A.S. Grm
- **date**: 22/11/2023
- **primer**: 03-2

<hr>

## Robni problem (ne-linearni 2. stopnje)

S pomočjo **metode končnih diferenc** reši ne-linearni robni problem

\begin{align}
    (2 - x^2)y'' - 2 x y' + \sin x \: y^2 & = x^2,\\
       y(-1) & = 1,\\
       y(1) & = -1.
\end{align}

<hr>

V zgornji enačbi nadomestitimo prvi in drugi odvod s končnimi diferencami

 $$(2-x_i^2) \frac{y_{i+1} - 2 y_i + y_{i-1}}{h^2} - 2x_i \frac{y_{i+1} - y_i}{2 h} + \sin x_i \: y_i^2 = x_i^2$$
	
Rešujemo sistem nelinearnih enačb $F(x,y_{i-1},y_i,y_{i+1}) = 0$ ($i=1,\dots,N$), s funkcijo
	
$$F(x_i,y_{i-1},y_i,y_{i+1}) = (2-x_i^2) \frac{y_{i+1} - 2 y_i + y_{i-1}}{h^2} - 2x_i \frac{y_{i+1} - y_i}{2 h} + \sin x_i \: y_i^2 - x_i^2,$$
	
kjer je upoštevan robni pogoj $y(-1) = y_0 = 1$ in $y(1) = y_{N+1} = -1$.

In [ ]:
import sys
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
import scipy.optimize as spo

# MatPlotLib set fonts
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['DejaVu Serif']

# MatPlotLib set LaTeX use
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{siunitx}'

In [ ]:
# Poiščemo vredost funkcije F=F(x,y)
def func(y, x, ys, ye, h, N):
    
    F = np.zeros(N) # optimizacijska funkcija

    F[0] = fF(x[0],h,ys,y[0],y[1])
    
    for i in range(1,N-1):
        F[i] = fF(x[i],h,y[i-1],y[i],y[i+1])

    F[N-1] = fF(x[N-1],h,y[N-2],y[N-1],ye)
    return F

def fF(x,h,y1,y2,y3):

    f = (2 - x**2) * (y3 - 2*y2 + y1)/(h**2) - 2*x * (y3 - y2)/(2*h) + np.sin(x)*y2**2 - x**2

    return f

In [ ]:
def solveBVP(xs, ys, xe, ye, N):

    # korak
    h = (xe - xs)/N
    k = (ye - ys)/(xe - xs)

    # rezerviramo prostor za vektorje
    x = np.zeros(N+1)
    y = np.zeros(N+1)

    # postavimo začetne vrednosti
    for i in range(N+1):
        x[i] = xs + i*h     # vmesne točke x_i
        y[i] = ys + k*(i*h) # inicializacija y_i

    # vmesne točke in vmesne vrednosti, ki jih iščemo
    x0 = x[1:N]
    y0 = y[1:N]
    nn = x0.size
    
    [sol,ds,cc,msg] = spo.fsolve(func,y0, args=(x0,ys,ye,h,nn), full_output=True)

    if cc == 1:
        print('Number of function calls:', ds['nfev'])
        print('norm(F):', np.linalg.norm(ds['fvec']))
        y[1:N] = sol
        return [x,y]
    else:
        print('ERROR:', msg)
        sys.exit()

In [ ]:
# Glavni program
N = 200
x0 = -1
xe = 1
y0 = 0 # leva roba vrednost, y(x0)
ye = 0 # desna robna vrednost, y (xe)

# reši začetni problem
[x,y] = solveBVP(x0, y0, xe, ye, N)

dydx = (y[1] - y[0])/(x[1] - x[0])
print('approx y\'=', dydx)

In [ ]:
fig, ax = mpl.subplots()
fig.suptitle(r'MKD metoda -- Ne-Linearni robni problem ($N={:d}$)'.format(N)) # Figure title

ax.plot(x,y)
ax.set_xlabel(r'$x_i$')
ax.set_ylabel(r'$y_i$')
ax.grid()

fig.tight_layout()
fig.savefig('02_primer-03-2.pdf')